# Hidden Partitioning in PySpark with Apache Iceberg

## Setup Environment

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from datetime import datetime, timedelta
import random
import time

In [ ]:
# Create a Spark session with Iceberg configuration
spark = SparkSession.builder \
    .appName("IcebergComplexSchemaEvolution") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local.type", "hadoop") \
    .config("spark.sql.catalog.local.warehouse", "/home/jovyan/iceberg/warehouse") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .getOrCreate()

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 38742)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/usr/local/spark/python/pyspark/accumulators.py", line 271, in accum_updates
    num_updates =

In [3]:
# Create a database if it doesn't exist
spark.sql("CREATE DATABASE IF NOT EXISTS local.db")
spark.sql("USE local.db")

DataFrame[]

## Understanding Hidden Partitioning

Hidden partitioning in Iceberg means:
1. Partitioning is defined in the table metadata
2. You don't need to include partition columns in your queries
3. Iceberg automatically prunes partitions based on filter conditions

## Example 1: Time-Based Hidden Partitioning

In [4]:
# Create a table with hidden partitioning by month
spark.sql("""
CREATE TABLE IF NOT EXISTS events (
    event_id bigint,
    event_time timestamp,
    user_id bigint,
    event_type string,
    details string
) USING iceberg
PARTITIONED BY (months(event_time))
""")

DataFrame[]

In [5]:
# Generate some sample data
data = []
for i in range(1, 1001):
    event_time = datetime(2023, 1, 1) + timedelta(days=random.randint(0, 364))
    data.append((
        i,
        event_time,
        random.randint(1, 100),
        random.choice(["click", "view", "purchase", "login"]),
        f"details_{i}"
    ))

In [6]:
# Create a DataFrame and write to the table
events_df = spark.createDataFrame(data, ["event_id", "event_time", "user_id", "event_type", "details"])
events_df.writeTo("local.db.events").append()

In [7]:
# Show the partitions (note the partition values are not actual columns)
spark.sql("SELECT * FROM local.db.events.partitions").show(truncate=False)

+---------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+----------------------+------------------------+
|partition|spec_id|record_count|file_count|total_data_file_size_in_bytes|position_delete_record_count|position_delete_file_count|equality_delete_record_count|equality_delete_file_count|last_updated_at       |last_updated_snapshot_id|
+---------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+----------------------+------------------------+
|{646}    |0      |80          |1         |2438                         |0                           |0                         |0                           |0                         |2025-07-11 20:05:44.68|3093749453819535185     |
|{647}    |0      |92          |1         |2523                 

## Querying with Hidden Partitioning

In [8]:
# Query with a time filter - Iceberg will automatically prune partitions
jan_events = spark.sql("""
SELECT * FROM events
WHERE event_time BETWEEN '2023-01-01' AND '2023-01-31'
""")

In [9]:
# Explain plan shows partition pruning
jan_events.explain(extended=True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter (('event_time >= 2023-01-01) AND ('event_time <= 2023-01-31))
   +- 'UnresolvedRelation [events], [], false

== Analyzed Logical Plan ==
event_id: bigint, event_time: timestamp, user_id: bigint, event_type: string, details: string
Project [event_id#103L, event_time#104, user_id#105L, event_type#106, details#107]
+- Filter ((event_time#104 >= cast(2023-01-01 as timestamp)) AND (event_time#104 <= cast(2023-01-31 as timestamp)))
   +- SubqueryAlias local.db.events
      +- RelationV2[event_id#103L, event_time#104, user_id#105L, event_type#106, details#107] local.db.events local.db.events

== Optimized Logical Plan ==
Filter (event_time#104 <= 2023-01-31 00:00:00)
+- RelationV2[event_id#103L, event_time#104, user_id#105L, event_type#106, details#107] local.db.events

== Physical Plan ==
*(1) Filter (event_time#104 <= 2023-01-31 00:00:00)
+- *(1) ColumnarToRow
   +- BatchScan local.db.events[event_id#103L, event_time#104, user_id#105L, event

In [10]:
# Compare with a query that doesn't benefit from pruning
all_events = spark.sql("SELECT * FROM events")
all_events.explain(extended=True)

== Parsed Logical Plan ==
'Project [*]
+- 'UnresolvedRelation [events], [], false

== Analyzed Logical Plan ==
event_id: bigint, event_time: timestamp, user_id: bigint, event_type: string, details: string
Project [event_id#123L, event_time#124, user_id#125L, event_type#126, details#127]
+- SubqueryAlias local.db.events
   +- RelationV2[event_id#123L, event_time#124, user_id#125L, event_type#126, details#127] local.db.events local.db.events

== Optimized Logical Plan ==
RelationV2[event_id#123L, event_time#124, user_id#125L, event_type#126, details#127] local.db.events

== Physical Plan ==
*(1) ColumnarToRow
+- BatchScan local.db.events[event_id#123L, event_time#124, user_id#125L, event_type#126, details#127] local.db.events (branch=null) [filters=, groupedBy=] RuntimeFilters: []



## Example 2: Bucket Partitioning

In [11]:
# Create a table with bucket partitioning
spark.sql("""
CREATE TABLE IF NOT EXISTS user_sessions (
    session_id bigint,
    user_id bigint,
    start_time timestamp,
    end_time timestamp,
    pages_visited int
) USING iceberg
PARTITIONED BY (bucket(16, user_id))
""")

DataFrame[]

In [12]:
# Generate sample data
sessions_data = []
for i in range(1, 1001):
    sessions_data.append((
        i,
        random.randint(1, 1000),
        datetime(2023, 1, 1) + timedelta(minutes=random.randint(0, 1440)),
        datetime(2023, 1, 1) + timedelta(minutes=random.randint(1440, 2880)),
        random.randint(1, 20)
    ))

In [13]:
sessions_df = spark.createDataFrame(sessions_data, 
                                  ["session_id", "user_id", "start_time", "end_time", "pages_visited"])
sessions_df.writeTo("local.db.user_sessions").append()

In [14]:
# Show bucket partitioning
spark.sql("SELECT * FROM local.db.user_sessions.partitions").show()

+---------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+--------------------+------------------------+
|partition|spec_id|record_count|file_count|total_data_file_size_in_bytes|position_delete_record_count|position_delete_file_count|equality_delete_record_count|equality_delete_file_count|     last_updated_at|last_updated_snapshot_id|
+---------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+--------------------+------------------------+
|      {8}|      0|          51|         1|                         2547|                           0|                         0|                           0|                         0|2025-07-11 20:08:...|     8921799163203253987|
|      {9}|      0|          75|         1|                         2745

## Querying Bucket Partitioned Table

In [15]:
# Query for a specific user - will only scan the relevant bucket
user_123_sessions = spark.sql("""
SELECT * FROM user_sessions
WHERE user_id = 123
""")

In [16]:
user_123_sessions.explain(extended=True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('user_id = 123)
   +- 'UnresolvedRelation [user_sessions], [], false

== Analyzed Logical Plan ==
session_id: bigint, user_id: bigint, start_time: timestamp, end_time: timestamp, pages_visited: int
Project [session_id#248L, user_id#249L, start_time#250, end_time#251, pages_visited#252]
+- Filter (user_id#249L = cast(123 as bigint))
   +- SubqueryAlias local.db.user_sessions
      +- RelationV2[session_id#248L, user_id#249L, start_time#250, end_time#251, pages_visited#252] local.db.user_sessions local.db.user_sessions

== Optimized Logical Plan ==
Filter (user_id#249L = 123)
+- RelationV2[session_id#248L, user_id#249L, start_time#250, end_time#251, pages_visited#252] local.db.user_sessions

== Physical Plan ==
*(1) Filter (user_id#249L = 123)
+- *(1) ColumnarToRow
   +- BatchScan local.db.user_sessions[session_id#248L, user_id#249L, start_time#250, end_time#251, pages_visited#252] local.db.user_sessions (branch=null) [filters=user_id IS

In [ ]:
# Compare with a query that can't use bucket pruning
all_sessions = spark.sql("SELECT * FROM user_sessions WHERE pages_visited > 10")
all_sessions.explain(extended=True)

## Example 3: Truncate Partitioning

In [ ]:
# Create a table with truncate partitioning
spark.sql("""
CREATE TABLE IF NOT EXISTS product_views (
    view_id bigint,
    product_sku string,
    user_id bigint,
    view_time timestamp,
    duration_seconds int
) USING iceberg
PARTITIONED BY (truncate(100, product_sku))
""")

In [ ]:
# Generate sample data
skus = [f"SKU-{i:05d}" for i in range(1, 1001)]
views_data = []
for i in range(1, 1001):
    views_data.append((
        i,
        random.choice(skus),
        random.randint(1, 1000),
        datetime(2023, 1, 1) + timedelta(seconds=random.randint(0, 86400)),
        random.randint(1, 300)
    ))

In [ ]:
views_df = spark.createDataFrame(views_data, 
                               ["view_id", "product_sku", "user_id", "view_time", "duration_seconds"])
views_df.writeTo("local.db.product_views").append()

In [ ]:
# Show truncate partitions
spark.sql("SELECT * FROM local.db.product_views.partitions").show(truncate=False)

## Querying Truncate Partitioned Table

In [ ]:
# Query for products in a specific SKU range
popular_skus = spark.sql("""
SELECT * FROM product_views
WHERE product_sku LIKE 'SKU-01%'
""")

In [ ]:
popular_skus.explain(extended=True)

## Advanced Example: Nested Partitioning

In [ ]:
# Create a table with nested partitioning
spark.sql("""
CREATE TABLE IF NOT EXISTS server_logs (
    log_id bigint,
    server_id string,
    event_time timestamp,
    log_level string,
    message string
) USING iceberg
PARTITIONED BY (days(event_time), bucket(8, server_id), log_level)
""")

In [ ]:
# Generate sample data
servers = [f"server-{i}" for i in ['a', 'b', 'c', 'd', 'e']]
levels = ["INFO", "WARN", "ERROR", "DEBUG"]
log_data = []
for i in range(1, 1001):
    log_data.append((
        i,
        random.choice(servers),
        datetime(2023, 1, 1) + timedelta(hours=random.randint(0, 24*30)),
        random.choice(levels),
        f"Log message {i}"
    ))

In [ ]:
logs_df = spark.createDataFrame(log_data, ["log_id", "server_id", "event_time", "log_level", "message"])
logs_df.writeTo("local.db.server_logs").append()

In [ ]:
# Show nested partitions
spark.sql("SELECT * FROM local.db.server_logs.partitions").show(truncate=False)

## Querying Nested Partitioned Table

In [ ]:
# Query that benefits from all partition levels
filtered_logs = spark.sql("""
SELECT * FROM server_logs
WHERE event_time BETWEEN '2023-01-15' AND '2023-01-16'
  AND server_id = 'server-a'
  AND log_level = 'ERROR'
""")

In [ ]:
filtered_logs.explain(extended=True)

In [ ]:
# Query that benefits from some partition levels
partial_filter_logs = spark.sql("""
SELECT * FROM server_logs
WHERE event_time BETWEEN '2023-01-15' AND '2023-01-16'
  AND log_level IN ('ERROR', 'WARN')
""")

In [ ]:
partial_filter_logs.explain(extended=True)

## Modifying Partitioning

In [ ]:
# Let's add a new partition field to our events table
spark.sql("""
ALTER TABLE events
ADD PARTITION FIELD hours(event_time)
""")

In [ ]:
# Show the updated partition spec
spark.sql("SELECT * FROM local.db.events.partitions").show(truncate=False)

In [ ]:
# Now writes will use both month and hour partitioning
new_events_data = []
for i in range(1001, 1101):
    event_time = datetime(2023, 6, 15) + timedelta(hours=random.randint(0, 23))
    new_events_data.append((
        i,
        event_time,
        random.randint(1, 100),
        random.choice(["click", "view", "purchase", "login"]),
        f"details_{i}"
    ))

In [ ]:
new_events_df = spark.createDataFrame(new_events_data, 
                                    ["event_id", "event_time", "user_id", "event_type", "details"])
new_events_df.writeTo("local.db.events").append()

In [ ]:
# Check the new partitions
spark.sql("SELECT * FROM local.db.events.partitions").show(truncate=False)

## Performance Comparison

In [ ]:
# With partition pruning
start_time = time.time()
pruned_query = spark.sql("""
SELECT COUNT(*) FROM events
WHERE event_time BETWEEN '2023-06-01' AND '2023-06-30'
""")
pruned_query.show()
print(f"Time with partition pruning: {time.time() - start_time:.2f} seconds")

In [ ]:
# Without partition pruning
start_time = time.time()
full_scan = spark.sql("""
SELECT COUNT(*) FROM events
WHERE event_type = 'click'
""")
full_scan.show()
print(f"Time without partition pruning: {time.time() - start_time:.2f} seconds")

## Best Practices

In [ ]:
# Check partition sizes
spark.sql("""
SELECT partition, record_count, file_count
FROM local.db.events.partitions
ORDER BY record_count DESC
""").show(truncate=False)

## Cleanup

In [ ]:
# Drop tables if needed
spark.sql("DROP TABLE IF EXISTS local.db.events")
spark.sql("DROP TABLE IF EXISTS local.db.user_sessions")
spark.sql("DROP TABLE IF EXISTS local.db.product_views")
spark.sql("DROP TABLE IF EXISTS local.db.server_logs")